In [ ]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [1]:
import requests
import pandas as pd
import re
import numpy
from google.cloud import storage

In [2]:
URL = 'http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv'
FILEPATH = './data/uci_bogota_raw.csv'

In [3]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [4]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 3 )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
def convert_int64(column:str):
    temp=ucis[column]
    i=0
    for row in temp:
        data=str(row) 
        search=re.findall('^[1-9]\.\w*',data)
        if(len(search)==1):
            #Select first item in list of re
            str1=search[0]
            
            #To float to aproximate 3 digits
            str1=float(str1)
            str1=round(str1,3)
            #Cast as str and replace '.'
            str1=str(str1).replace('.','')
            if(len(str1)==3):
                str1=str1+'0'
                
            #Replace in the column
            temp[i]=str1
            temp[i]=temp[i].astype(numpy.int64)
            
        else:
            row=float(data)
            temp[i]=int(row)
            temp[i]=temp[i].astype(numpy.int64)
        i+=1
    
    ucis[column]=temp.astype(numpy.int64)

convert_int64('Camas UCI ocupadas Covid-19')
convert_int64('Camas UCI Disponibles COVID 19')

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py

In [6]:
ucis.shape

(244, 4)

In [7]:
ucis.dtypes

Fecha                             object
Camas UCI ocupadas Covid-19        int64
Camas UCI Disponibles COVID 19     int64
Ocupación UCI COVID 19            object
dtype: object

In [8]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%Y' )

In [9]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [10]:
ucis.dtypes

Fecha                             datetime64[ns]
Camas UCI ocupadas Covid-19                int64
Camas UCI Disponibles COVID 19             int64
Ocupación UCI COVID 19                    object
% Ocupación                              float64
dtype: object

In [11]:
del ucis[ 'Ocupación UCI COVID 19' ]

In [12]:
ucis.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [13]:
ucis.tail( 20 )

,Fecha,Camas UCI ocupadas Covid-19,Camas UCI Disponibles COVID 19,Ocupacion
224,2020-11-19,790,1628,0.485258
225,2020-11-20,779,1625,0.479385
226,2020-11-21,797,1620,0.491975
227,2020-11-22,822,1621,0.507094
228,2020-11-23,805,1618,0.497528
229,2020-11-24,834,1619,0.515133
230,2020-11-25,806,1621,0.497224
231,2020-11-26,795,1621,0.490438
232,2020-11-27,769,1599,0.480926
233,2020-11-28,812,1613,0.503410


In [14]:
#ucis.to_csv( './data/uci_bogota.csv', index = False )

In [15]:
ucis[ [ 'Fecha', 'Camas UCI ocupadas Covid-19' ] ].to_csv( './alerts/data/uci_bogota.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=ucis.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('uci_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')